In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from spmf import Spmf
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import ast
pd.options.plotting.backend = "plotly"
import plotly.express as px
# from chart_studio import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.subplots as sp
pd.options.display.max_colwidth = None
pd.set_option('display.max_rows', 70)
from pandas.plotting import table 
import nltk
import seaborn as sns
%matplotlib inline
import random
import matplotlib.gridspec as grid
import matplotlib
from sklearn.decomposition import NMF
from ast import literal_eval

We start by importing the user groups for each of the eight components.

In [2]:
top_500_factor_0_loadings=pd.read_csv("top500_users_facebook_component.csv").user_id
top_500_factor_1_loadings=pd.read_csv("top500_users_google_component.csv").user_id
top_500_factor_2_loadings=pd.read_csv("top500_users_weather_component.csv").user_id
top_500_factor_3_loadings=pd.read_csv("top500_users_social_media_multimedia_component.csv").user_id
top_500_factor_4_loadings=pd.read_csv("top500_users_traditional_communication_component.csv").user_id
top_500_factor_5_loadings=pd.read_csv("top500_users_chrome_component.csv").user_id
top_500_factor_6_loadings=pd.read_csv("top500_users_whatsapp_component.csv").user_id
top_500_factor_R_loadings=pd.read_csv("top500_users_email_component.csv").user_id

In [3]:
data=pd.read_csv("sorted_coded_session_293s_merged.csv")

We take a look at the most popular apps within the groups.

In [4]:
data[data.user_id.isin(top_500_factor_0_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.facebook.katana               4602833
com.facebook.orca                 1025367
com.whatsapp                       972763
com.android.chrome                 945629
com.sonyericsson.conversations     864159
dtype: int64

In [5]:
data[data.user_id.isin(top_500_factor_1_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.google.android.googlequicksearchbox    2111952
com.android.chrome                         1334558
com.facebook.katana                         857655
com.whatsapp                                750769
com.sonyericsson.conversations              637747
dtype: int64

In [6]:
data[data.user_id.isin(top_500_factor_2_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.facebook.katana                        1370416
com.whatsapp                                996738
com.android.chrome                          902091
com.sonyericsson.conversations              796328
com.google.android.googlequicksearchbox     495568
dtype: int64

In [7]:
data[data.user_id.isin(top_500_factor_3_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.snapchat.android     3162694
com.instagram.android    2313139
com.facebook.katana      2230080
com.facebook.orca        2079059
com.whatsapp             1209774
dtype: int64

In [8]:
data[data.user_id.isin(top_500_factor_4_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.sonyericsson.conversations              1428009
com.facebook.katana                          764845
com.android.chrome                           541998
com.sonyericsson.android.socialphonebook     451154
com.android.incallui                         442135
dtype: int64

In [9]:
data[data.user_id.isin(top_500_factor_5_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.android.chrome                2836496
com.whatsapp                      1104443
com.facebook.katana                950079
com.sonyericsson.conversations     746632
com.facebook.orca                  407761
dtype: int64

In [10]:
data[data.user_id.isin(top_500_factor_6_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.whatsapp                               5247808
com.facebook.katana                        1614664
com.android.chrome                         1180628
com.instagram.android                       805148
com.google.android.googlequicksearchbox     638177
dtype: int64

In [11]:
data[data.user_id.isin(top_500_factor_R_loadings)].groupby('package_name').size().nlargest(5)

package_name
com.sonymobile.email              1634651
com.facebook.katana               1143305
com.android.chrome                1023942
com.whatsapp                       807981
com.sonyericsson.conversations     782363
dtype: int64

As expected we see the top apps are different, and in accordance with the apps central in the charaterization of the components.

For the N-gram the sessions of length one are excluded.

In [4]:
data=data.drop(columns=['start_time','end_time','time_spend','sequence_number','sequence_Start_time','sequence_end_time'])

In [5]:
data=pd.merge(data,data.groupby(['user_id','session']).size().reset_index(),on=['user_id','session']).rename(columns={0:'session_length'})

In [6]:
data=data[data.session_length!=1]

The corpus for each user group is now constructed.

In [15]:
corpus_top_500_factor_0_loadings=data.loc[data.user_id.isin(top_500_factor_0_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)
corpus_top_500_factor_1_loadings=data.loc[data.user_id.isin(top_500_factor_1_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)
corpus_top_500_factor_2_loadings=data.loc[data.user_id.isin(top_500_factor_2_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)
corpus_top_500_factor_3_loadings=data.loc[data.user_id.isin(top_500_factor_3_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)
corpus_top_500_factor_4_loadings=data.loc[data.user_id.isin(top_500_factor_4_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)
corpus_top_500_factor_5_loadings=data.loc[data.user_id.isin(top_500_factor_5_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)
corpus_top_500_factor_6_loadings=data.loc[data.user_id.isin(top_500_factor_6_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)
corpus_top_500_factor_R_loadings=data.loc[data.user_id.isin(top_500_factor_R_loadings)].reset_index().groupby(['user_id','session']).package_name.apply(' '.join)

We take a look at the number of sessions, with more than one app, in each group.

In [16]:
print("Number sessions with more than one app")
print("----------------------------------------")
print("Facebook component:",corpus_top_500_factor_0_loadings.shape[0])
print("Chrome component:",corpus_top_500_factor_1_loadings.shape[0])
print("Weather component:",corpus_top_500_factor_2_loadings.shape[0])
print("Facebook component:",corpus_top_500_factor_3_loadings.shape[0])
print("Social media - multimedia component:",corpus_top_500_factor_4_loadings.shape[0])
print("Tradition communication component:",corpus_top_500_factor_5_loadings.shape[0])
print("Whatsapp component:",corpus_top_500_factor_6_loadings.shape[0])
print("Email component:",corpus_top_500_factor_R_loadings.shape[0])

Number sessions with more than one app
----------------------------------------
Facebook component: 2608785
Chrome component: 2056107
Weather component: 1938907
Facebook component: 3136026
Social media - multimedia component: 1444760
Tradition communication component: 2470782
Whatsapp component: 2970977
Email component: 2149805


The number above is much lower that previously, so a higher min. support. is needed.

In [17]:
print("Number sessions needed for 1% support")
print("----------------------------------------")
print("Facebook component:",np.ceil(corpus_top_500_factor_0_loadings.shape[0]/100*1))
print("Chrome component:",np.ceil(corpus_top_500_factor_1_loadings.shape[0]/100*1))
print("Weather component:",np.ceil(corpus_top_500_factor_2_loadings.shape[0]/100*1))
print("Facebook component:",np.ceil(corpus_top_500_factor_3_loadings.shape[0]/100*1))
print("Social media - multimedia component:",np.ceil(corpus_top_500_factor_4_loadings.shape[0]/100*1))
print("Tradition communication component:",np.ceil(corpus_top_500_factor_5_loadings.shape[0]/100*1))
print("Whatsapp component:",np.ceil(corpus_top_500_factor_6_loadings.shape[0]/100*1))
print("Email component:",np.ceil(corpus_top_500_factor_R_loadings.shape[0]/100*1))

Number sessions needed for 1% support
----------------------------------------
Facebook component: 26088.0
Chrome component: 20562.0
Weather component: 19390.0
Facebook component: 31361.0
Social media - multimedia component: 14448.0
Tradition communication component: 24708.0
Whatsapp component: 29710.0
Email component: 21499.0


In [18]:
min_sup_0=np.ceil(corpus_top_500_factor_0_loadings.shape[0]/100*1)
min_sup_1=np.ceil(corpus_top_500_factor_1_loadings.shape[0]/100*1)
min_sup_2=np.ceil(corpus_top_500_factor_2_loadings.shape[0]/100*1)
min_sup_3=np.ceil(corpus_top_500_factor_3_loadings.shape[0]/100*1)
min_sup_4=np.ceil(corpus_top_500_factor_4_loadings.shape[0]/100*1)
min_sup_5=np.ceil(corpus_top_500_factor_5_loadings.shape[0]/100*1)
min_sup_6=np.ceil(corpus_top_500_factor_6_loadings.shape[0]/100*1)
min_sup_r=np.ceil(corpus_top_500_factor_R_loadings.shape[0]/100*1)

# Frequent apps / 1-gram

In [19]:
vectorizer_1_f0 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_f0=vectorizer_1_f0.fit_transform(corpus_top_500_factor_0_loadings.values)
Facebook_component_1gram=pd.Series(vectorizer_1_f0.get_feature_names_out()[np.where((vectorization_1_f0>0).sum(axis=0)>=min_sup_0)[1]])
Facebook_component_1gram.to_csv("1gram_facebook_component_minsup1.csv")
print("Number of frequent apps:",len(Facebook_component_1gram))
Facebook_component_1gram

Number of frequent apps: 27


0                           bbc.mobile.news.uk
1            com.amazon.mShop.android.shopping
2                           com.android.chrome
3                         com.android.incallui
4                            com.android.phone
5                              com.ebay.mobile
6                          com.facebook.katana
7                            com.facebook.orca
8                 com.google.android.apps.maps
9                        com.google.android.gm
10     com.google.android.googlequicksearchbox
11                  com.google.android.youtube
12                       com.instagram.android
13                com.microsoft.office.outlook
14                   com.nianticlabs.pokemongo
15                        com.snapchat.android
16                       com.sony.nfx.app.sfrc
17                      com.sonyericsson.album
18    com.sonyericsson.android.socialphonebook
19              com.sonyericsson.conversations
20                      com.sonyericsson.music
21           

In [20]:
vectorizer_1_f1 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_f1=vectorizer_1_f1.fit_transform(corpus_top_500_factor_1_loadings.values)
google_component_1gram=pd.Series(vectorizer_1_f1.get_feature_names_out()[np.where((vectorization_1_f1>0).sum(axis=0)>=min_sup_1)[1]])
google_component_1gram.to_csv("1gram_google_component_minsup1.csv")
print("Number of frequent apps:",len(google_component_1gram))
google_component_1gram

Number of frequent apps: 34


0                           bbc.mobile.news.uk
1                           bbc.mobile.weather
2            com.amazon.mShop.android.shopping
3                           com.android.chrome
4                         com.android.incallui
5                            com.android.phone
6                         com.dailymail.online
7                              com.ebay.mobile
8                          com.facebook.katana
9                            com.facebook.orca
10                com.google.android.apps.docs
11                com.google.android.apps.maps
12           com.google.android.apps.messaging
13              com.google.android.apps.photos
14                 com.google.android.calendar
15                       com.google.android.gm
16     com.google.android.googlequicksearchbox
17                  com.google.android.youtube
18                       com.instagram.android
19                com.microsoft.office.outlook
20                   com.nianticlabs.pokemongo
21           

In [21]:
vectorizer_1_f2 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_f2=vectorizer_1_f2.fit_transform(corpus_top_500_factor_2_loadings.values)
Weather_component_1gram=pd.Series(vectorizer_1_f2.get_feature_names_out()[np.where((vectorization_1_f2>0).sum(axis=0)>=min_sup_2)[1]])
Weather_component_1gram.to_csv("1gram_weather_component_minsup1.csv")
print("Number of frequent apps:",len(Weather_component_1gram))
Weather_component_1gram

Number of frequent apps: 31


0                           bbc.mobile.news.uk
1            com.amazon.mShop.android.shopping
2                           com.android.chrome
3                            com.android.email
4                         com.android.incallui
5                            com.android.phone
6                              com.ebay.mobile
7                          com.facebook.katana
8                            com.facebook.orca
9                 com.google.android.apps.docs
10                com.google.android.apps.maps
11                       com.google.android.gm
12     com.google.android.googlequicksearchbox
13                  com.google.android.youtube
14                       com.instagram.android
15                com.microsoft.office.outlook
16                   com.nianticlabs.pokemongo
17                               com.pinterest
18                        com.snapchat.android
19                       com.sony.nfx.app.sfrc
20                      com.sonyericsson.album
21    com.son

In [22]:
vectorizer_1_f3 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_f3=vectorizer_1_f3.fit_transform(corpus_top_500_factor_3_loadings.values)
social_multimedia_component_1gram=pd.Series(vectorizer_1_f3.get_feature_names_out()[np.where((vectorization_1_f3>0).sum(axis=0)>=min_sup_3)[1]])
social_multimedia_component_1gram.to_csv("1gram_social_multimedia_component_minsup1.csv")
print("Number of frequent apps:",len(social_multimedia_component_1gram))
social_multimedia_component_1gram

Number of frequent apps: 27


0                           bbc.mobile.news.uk
1                           com.android.chrome
2                         com.android.incallui
3                            com.android.phone
4                          com.facebook.katana
5                            com.facebook.orca
6                 com.google.android.apps.maps
7            com.google.android.apps.messaging
8                        com.google.android.gm
9      com.google.android.googlequicksearchbox
10                  com.google.android.youtube
11                       com.instagram.android
12                com.microsoft.office.outlook
13                   com.nianticlabs.pokemongo
14                               com.pinterest
15                        com.snapchat.android
16                       com.sony.nfx.app.sfrc
17                      com.sonyericsson.album
18    com.sonyericsson.android.socialphonebook
19              com.sonyericsson.conversations
20                      com.sonyericsson.music
21           

In [23]:
vectorizer_1_f4 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_f4=vectorizer_1_f4.fit_transform(corpus_top_500_factor_4_loadings.values)
traditional_component_1gram=pd.Series(vectorizer_1_f4.get_feature_names_out()[np.where((vectorization_1_f4>0).sum(axis=0)>=min_sup_4)[1]])
traditional_component_1gram.to_csv("1gram_tradition_component_minsup1.csv")
print("Number of frequent apps:",len(traditional_component_1gram))
traditional_component_1gram

Number of frequent apps: 25


0                           bbc.mobile.news.uk
1                           com.android.chrome
2                            com.android.email
3                         com.android.incallui
4                            com.android.phone
5                              com.ebay.mobile
6                          com.facebook.katana
7                            com.facebook.orca
8                 com.google.android.apps.maps
9                        com.google.android.gm
10     com.google.android.googlequicksearchbox
11                  com.google.android.youtube
12                       com.instagram.android
13                com.microsoft.office.outlook
14                        com.snapchat.android
15                       com.sony.nfx.app.sfrc
16                      com.sonyericsson.album
17    com.sonyericsson.android.socialphonebook
18              com.sonyericsson.conversations
19                      com.sonyericsson.music
20                     com.sonymobile.calendar
21           

In [24]:
vectorizer_1_f5 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_f5=vectorizer_1_f5.fit_transform(corpus_top_500_factor_5_loadings.values)
chrome_component_1gram=pd.Series(vectorizer_1_f5.get_feature_names_out()[np.where((vectorization_1_f5>0).sum(axis=0)>=min_sup_5)[1]])
chrome_component_1gram.to_csv("1gram_chrome_component_minsup1.csv")
print("Number of frequent apps:",len(chrome_component_1gram))
chrome_component_1gram

Number of frequent apps: 31


0                           bbc.mobile.news.uk
1            com.amazon.mShop.android.shopping
2                           com.android.chrome
3                         com.android.incallui
4                            com.android.phone
5                              com.ebay.mobile
6                          com.facebook.katana
7                            com.facebook.orca
8                 com.google.android.apps.docs
9                 com.google.android.apps.maps
10           com.google.android.apps.messaging
11                 com.google.android.calendar
12                       com.google.android.gm
13     com.google.android.googlequicksearchbox
14                  com.google.android.youtube
15                       com.instagram.android
16                com.microsoft.office.outlook
17                   com.nianticlabs.pokemongo
18                        com.snapchat.android
19                       com.sony.nfx.app.sfrc
20                      com.sonyericsson.album
21    com.son

In [25]:
vectorizer_1_f6 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_f6=vectorizer_1_f6.fit_transform(corpus_top_500_factor_6_loadings.values)
whatsapp_component_1gram=pd.Series(vectorizer_1_f5.get_feature_names_out()[np.where((vectorization_1_f5>0).sum(axis=0)>=min_sup_5)[1]])
whatsapp_component_1gram.to_csv("1gram_whatsapp_component_minsup1.csv")
print("Number of frequent apps:",len(whatsapp_component_1gram))
whatsapp_component_1gram

Number of frequent apps: 31


0                           bbc.mobile.news.uk
1            com.amazon.mShop.android.shopping
2                           com.android.chrome
3                         com.android.incallui
4                            com.android.phone
5                              com.ebay.mobile
6                          com.facebook.katana
7                            com.facebook.orca
8                 com.google.android.apps.docs
9                 com.google.android.apps.maps
10           com.google.android.apps.messaging
11                 com.google.android.calendar
12                       com.google.android.gm
13     com.google.android.googlequicksearchbox
14                  com.google.android.youtube
15                       com.instagram.android
16                com.microsoft.office.outlook
17                   com.nianticlabs.pokemongo
18                        com.snapchat.android
19                       com.sony.nfx.app.sfrc
20                      com.sonyericsson.album
21    com.son

In [26]:
vectorizer_1_fr = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization_1_fr=vectorizer_1_fr.fit_transform(corpus_top_500_factor_R_loadings.values)
email_component_1gram=pd.Series(vectorizer_1_fr.get_feature_names_out()[np.where((vectorization_1_fr>0).sum(axis=0)>=min_sup_r)[1]])
email_component_1gram.to_csv("1gram_email_component_minsup1.csv")
print("Number of frequent apps:",len(email_component_1gram))
email_component_1gram

Number of frequent apps: 34


0                           bbc.mobile.news.uk
1                           bbc.mobile.weather
2            com.amazon.mShop.android.shopping
3                           com.android.chrome
4                            com.android.email
5                         com.android.incallui
6                              com.ebay.mobile
7                          com.facebook.katana
8                            com.facebook.orca
9                      com.fitbit.FitbitMobile
10                com.google.android.apps.docs
11                com.google.android.apps.maps
12           com.google.android.apps.messaging
13              com.google.android.apps.photos
14                 com.google.android.calendar
15                       com.google.android.gm
16     com.google.android.googlequicksearchbox
17                  com.google.android.youtube
18                       com.instagram.android
19                        com.linkedin.android
20                com.microsoft.office.outlook
21           

# 2-grams

In [27]:
vectorizer_2_f0 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_f0=vectorizer_2_f0.fit_transform(corpus_top_500_factor_0_loadings.values)
Facebook_component_2gram=pd.Series(vectorizer_2_f0.get_feature_names_out()[np.where((vectorization_2_f0>0).sum(axis=0)>=min_sup_0)[1]])
Facebook_component_2gram.to_csv("2gram_facebook_component_minsup1.csv")
print("Number of frequent apps:",len(Facebook_component_2gram))
Facebook_component_2gram

Number of frequent apps: 43


0                            com.android.chrome com.facebook.katana
1                              com.android.chrome com.facebook.orca
2        com.android.chrome com.google.android.googlequicksearchbox
3                 com.android.chrome com.sonyericsson.conversations
4                                   com.android.chrome com.whatsapp
5                          com.android.incallui com.facebook.katana
6     com.android.incallui com.sonyericsson.android.socialphonebook
7                            com.facebook.katana com.android.chrome
8                          com.facebook.katana com.android.incallui
9                             com.facebook.katana com.facebook.orca
10                 com.facebook.katana com.google.android.apps.maps
11                        com.facebook.katana com.google.android.gm
12      com.facebook.katana com.google.android.googlequicksearchbox
13                   com.facebook.katana com.google.android.youtube
14                        com.facebook.katana co

In [28]:
vectorizer_2_f1 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_f1=vectorizer_2_f1.fit_transform(corpus_top_500_factor_1_loadings.values)
google_component_2gram=pd.Series(vectorizer_2_f1.get_feature_names_out()[np.where((vectorization_2_f1>0).sum(axis=0)>=min_sup_1)[1]])
google_component_2gram.to_csv("2gram_google_component_minsup1.csv")
print("Number of frequent apps:",len(google_component_2gram))
google_component_2gram

Number of frequent apps: 35


0                                     com.android.chrome com.facebook.katana
1                                   com.android.chrome com.google.android.gm
2                 com.android.chrome com.google.android.googlequicksearchbox
3                          com.android.chrome com.sonyericsson.conversations
4                                            com.android.chrome com.whatsapp
5              com.android.incallui com.sonyericsson.android.socialphonebook
6                                     com.facebook.katana com.android.chrome
7                                      com.facebook.katana com.facebook.orca
8                com.facebook.katana com.google.android.googlequicksearchbox
9                         com.facebook.katana com.sonyericsson.conversations
10                                          com.facebook.katana com.whatsapp
11                                     com.facebook.orca com.facebook.katana
12                 com.facebook.orca com.google.android.googlequicksearchbox

In [29]:
vectorizer_2_f2 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_f2=vectorizer_2_f2.fit_transform(corpus_top_500_factor_2_loadings.values)
Weather_component_2gram=pd.Series(vectorizer_2_f2.get_feature_names_out()[np.where((vectorization_2_f2>0).sum(axis=0)>=min_sup_2)[1]])
Weather_component_2gram.to_csv("2gram_weather_component_minsup1.csv")
print("Number of frequent apps:",len(Weather_component_2gram))
Weather_component_2gram

Number of frequent apps: 42


0                                      com.android.chrome com.facebook.katana
1                  com.android.chrome com.google.android.googlequicksearchbox
2                           com.android.chrome com.sonyericsson.conversations
3                                             com.android.chrome com.whatsapp
4               com.android.incallui com.sonyericsson.android.socialphonebook
5                         com.android.incallui com.sonyericsson.conversations
6                                      com.facebook.katana com.android.chrome
7                                       com.facebook.katana com.facebook.orca
8                 com.facebook.katana com.google.android.googlequicksearchbox
9                                   com.facebook.katana com.instagram.android
10                                   com.facebook.katana com.snapchat.android
11                         com.facebook.katana com.sonyericsson.conversations
12                                   com.facebook.katana com.son

In [30]:
vectorizer_2_f3 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_f3=vectorizer_2_f3.fit_transform(corpus_top_500_factor_3_loadings.values)
social_multimedia_component_2gram=pd.Series(vectorizer_2_f3.get_feature_names_out()[np.where((vectorization_2_f3>0).sum(axis=0)>=min_sup_3)[1]])
social_multimedia_component_2gram.to_csv("2gram_social_multimedia_component_minsup1.csv")
print("Number of frequent apps:",len(social_multimedia_component_2gram))
social_multimedia_component_2gram

Number of frequent apps: 74


0                         com.android.chrome com.facebook.katana
1                           com.android.chrome com.facebook.orca
2     com.android.chrome com.google.android.googlequicksearchbox
3                  com.android.chrome com.google.android.youtube
4                       com.android.chrome com.instagram.android
                                 ...                            
69                               com.whatsapp com.android.chrome
70                              com.whatsapp com.facebook.katana
71                                com.whatsapp com.facebook.orca
72                            com.whatsapp com.instagram.android
73                             com.whatsapp com.snapchat.android
Length: 74, dtype: object

In [31]:
vectorizer_2_f4 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_f4=vectorizer_2_f4.fit_transform(corpus_top_500_factor_4_loadings.values)
traditional_component_2gram=pd.Series(vectorizer_2_f4.get_feature_names_out()[np.where((vectorization_2_f4>0).sum(axis=0)>=min_sup_4)[1]])
traditional_component_2gram.to_csv("2gram_tradition_component_minsup1.csv")
print("Number of frequent apps:",len(traditional_component_2gram))
traditional_component_2gram

Number of frequent apps: 37


0                                      com.android.chrome com.facebook.katana
1                  com.android.chrome com.google.android.googlequicksearchbox
2                           com.android.chrome com.sonyericsson.conversations
3                                     com.android.chrome com.sonymobile.email
4                                             com.android.chrome com.whatsapp
5                                    com.android.incallui com.facebook.katana
6               com.android.incallui com.sonyericsson.android.socialphonebook
7                         com.android.incallui com.sonyericsson.conversations
8                                      com.facebook.katana com.android.chrome
9                                       com.facebook.katana com.facebook.orca
10                com.facebook.katana com.google.android.googlequicksearchbox
11                         com.facebook.katana com.sonyericsson.conversations
12                                   com.facebook.katana com.son

In [32]:
vectorizer_2_f5 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_f5=vectorizer_2_f5.fit_transform(corpus_top_500_factor_5_loadings.values)
chrome_component_2gram=pd.Series(vectorizer_2_f5.get_feature_names_out()[np.where((vectorization_2_f5>0).sum(axis=0)>=min_sup_5)[1]])
chrome_component_2gram.to_csv("2gram_chrome_component_minsup1.csv")
print("Number of frequent apps:",len(chrome_component_2gram))
chrome_component_2gram

Number of frequent apps: 35


0                            com.android.chrome com.facebook.katana
1                              com.android.chrome com.facebook.orca
2                   com.android.chrome com.google.android.apps.maps
3                          com.android.chrome com.google.android.gm
4        com.android.chrome com.google.android.googlequicksearchbox
5                     com.android.chrome com.google.android.youtube
6                          com.android.chrome com.instagram.android
7                         com.android.chrome com.sonyericsson.album
8                 com.android.chrome com.sonyericsson.conversations
9                           com.android.chrome com.sonymobile.email
10                           com.android.chrome com.twitter.android
11                                  com.android.chrome com.whatsapp
12    com.android.incallui com.sonyericsson.android.socialphonebook
13                           com.facebook.katana com.android.chrome
14                            com.facebook.katan

In [33]:
vectorizer_2_f6 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_f6=vectorizer_2_f6.fit_transform(corpus_top_500_factor_6_loadings.values)
whatsapp_component_2gram=pd.Series(vectorizer_2_f5.get_feature_names_out()[np.where((vectorization_2_f5>0).sum(axis=0)>=min_sup_5)[1]])
whatsapp_component_2gram.to_csv("2gram_whatsapp_component_minsup1.csv")
print("Number of frequent apps:",len(whatsapp_component_2gram))
whatsapp_component_2gram

Number of frequent apps: 35


0                            com.android.chrome com.facebook.katana
1                              com.android.chrome com.facebook.orca
2                   com.android.chrome com.google.android.apps.maps
3                          com.android.chrome com.google.android.gm
4        com.android.chrome com.google.android.googlequicksearchbox
5                     com.android.chrome com.google.android.youtube
6                          com.android.chrome com.instagram.android
7                         com.android.chrome com.sonyericsson.album
8                 com.android.chrome com.sonyericsson.conversations
9                           com.android.chrome com.sonymobile.email
10                           com.android.chrome com.twitter.android
11                                  com.android.chrome com.whatsapp
12    com.android.incallui com.sonyericsson.android.socialphonebook
13                           com.facebook.katana com.android.chrome
14                            com.facebook.katan

In [34]:
vectorizer_2_fr = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2),lowercase=False)
vectorization_2_fr=vectorizer_2_fr.fit_transform(corpus_top_500_factor_R_loadings.values)
email_component_2gram=pd.Series(vectorizer_2_fr.get_feature_names_out()[np.where((vectorization_2_fr>0).sum(axis=0)>=min_sup_r)[1]])
email_component_2gram.to_csv("2gram_email_component_minsup1.csv")
print("Number of frequent apps:",len(email_component_2gram))
email_component_2gram

Number of frequent apps: 33


0                                     com.android.chrome com.facebook.katana
1                 com.android.chrome com.google.android.googlequicksearchbox
2                          com.android.chrome com.sonyericsson.conversations
3                                    com.android.chrome com.sonymobile.email
4                                            com.android.chrome com.whatsapp
5              com.android.incallui com.sonyericsson.android.socialphonebook
6                                     com.facebook.katana com.android.chrome
7                                      com.facebook.katana com.facebook.orca
8                com.facebook.katana com.google.android.googlequicksearchbox
9                         com.facebook.katana com.sonyericsson.conversations
10                                  com.facebook.katana com.sonymobile.email
11                                          com.facebook.katana com.whatsapp
12                                     com.facebook.orca com.facebook.katana

# 3-grams

In [35]:
vectorizer_3_f0 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_f0=vectorizer_3_f0.fit_transform(corpus_top_500_factor_0_loadings.values)
Facebook_component_3gram=pd.Series(vectorizer_3_f0.get_feature_names_out()[np.where((vectorization_3_f0>0).sum(axis=0)>=min_sup_0)[1]])
Facebook_component_3gram.to_csv("3gram_facebook_component_minsup1.csv")
print("Number of frequent apps:",len(Facebook_component_3gram))
Facebook_component_3gram

Number of frequent apps: 12


0                                              com.android.chrome com.facebook.katana com.android.chrome
1                                             com.facebook.katana com.android.chrome com.facebook.katana
2                                              com.facebook.katana com.facebook.orca com.facebook.katana
3                         com.facebook.katana com.google.android.googlequicksearchbox com.android.chrome
4                                          com.facebook.katana com.instagram.android com.facebook.katana
5                                 com.facebook.katana com.sonyericsson.conversations com.facebook.katana
6                                                   com.facebook.katana com.whatsapp com.facebook.katana
7                                                com.facebook.orca com.facebook.katana com.facebook.orca
8                         com.google.android.googlequicksearchbox com.android.chrome com.facebook.katana
9     com.google.android.googlequicksearchbox com.andro

In [36]:
vectorizer_3_f1 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_f1=vectorizer_3_f1.fit_transform(corpus_top_500_factor_1_loadings.values)
google_component_3gram=pd.Series(vectorizer_3_f1.get_feature_names_out()[np.where((vectorization_3_f1>0).sum(axis=0)>=min_sup_1)[1]])
google_component_3gram.to_csv("3gram_google_component_minsup1.csv")
print("Number of frequent apps:",len(google_component_3gram))
google_component_3gram

Number of frequent apps: 9


0                          com.android.chrome com.google.android.googlequicksearchbox com.android.chrome
1                         com.android.chrome com.google.android.googlequicksearchbox com.facebook.katana
2                         com.facebook.katana com.google.android.googlequicksearchbox com.android.chrome
3                         com.google.android.googlequicksearchbox com.android.chrome com.facebook.katana
4     com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox
5                                com.google.android.googlequicksearchbox com.android.chrome com.whatsapp
6    com.google.android.googlequicksearchbox com.facebook.katana com.google.android.googlequicksearchbox
7              com.sonyericsson.conversations com.google.android.googlequicksearchbox com.android.chrome
8                                com.whatsapp com.google.android.googlequicksearchbox com.android.chrome
dtype: object

In [37]:
vectorizer_3_f2 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_f2=vectorizer_3_f2.fit_transform(corpus_top_500_factor_3_loadings.values)
Weather_component_3gram=pd.Series(vectorizer_3_f2.get_feature_names_out()[np.where((vectorization_3_f2>0).sum(axis=0)>=min_sup_2)[1]])
Weather_component_3gram.to_csv("3gram_weather_component_minsup1.csv")
print("Number of frequent apps:",len(Weather_component_3gram))
Weather_component_3gram

Number of frequent apps: 47


0                                              com.facebook.katana com.facebook.orca com.facebook.katana
1                                            com.facebook.katana com.facebook.orca com.instagram.android
2                                             com.facebook.katana com.facebook.orca com.snapchat.android
3                                          com.facebook.katana com.instagram.android com.facebook.katana
4                                         com.facebook.katana com.instagram.android com.snapchat.android
5                                           com.facebook.katana com.snapchat.android com.facebook.katana
6                                             com.facebook.katana com.snapchat.android com.facebook.orca
7                                         com.facebook.katana com.snapchat.android com.instagram.android
8                                                com.facebook.orca com.facebook.katana com.facebook.orca
9                                            com.facebo

In [38]:
vectorizer_3_f3 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_f3=vectorizer_3_f3.fit_transform(corpus_top_500_factor_3_loadings.values)
social_multimedia_component_3gram=pd.Series(vectorizer_3_f3.get_feature_names_out()[np.where((vectorization_3_f3>0).sum(axis=0)>=min_sup_3)[1]])
social_multimedia_component_3gram.to_csv("3gram_social_multimedia_component_minsup1.csv")
print("Number of frequent apps:",len(social_multimedia_component_3gram))
social_multimedia_component_3gram

Number of frequent apps: 25


0            com.facebook.katana com.facebook.orca com.facebook.katana
1           com.facebook.katana com.facebook.orca com.snapchat.android
2        com.facebook.katana com.instagram.android com.facebook.katana
3       com.facebook.katana com.instagram.android com.snapchat.android
4         com.facebook.katana com.snapchat.android com.facebook.katana
5       com.facebook.katana com.snapchat.android com.instagram.android
6              com.facebook.orca com.facebook.katana com.facebook.orca
7           com.facebook.orca com.facebook.katana com.snapchat.android
8            com.facebook.orca com.instagram.android com.facebook.orca
9           com.facebook.orca com.snapchat.android com.facebook.katana
10            com.facebook.orca com.snapchat.android com.facebook.orca
11     com.instagram.android com.facebook.katana com.instagram.android
12      com.instagram.android com.facebook.katana com.snapchat.android
13      com.instagram.android com.snapchat.android com.facebook.katana
14    

In [39]:
vectorizer_3_f4 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_f4=vectorizer_3_f4.fit_transform(corpus_top_500_factor_4_loadings.values)
traditional_component_3gram=pd.Series(vectorizer_3_f4.get_feature_names_out()[np.where((vectorization_3_f4>0).sum(axis=0)>=min_sup_4)[1]])
traditional_component_3gram.to_csv("3gram_tradition_component_minsup1.csv")
print("Number of frequent apps:",len(traditional_component_3gram))
traditional_component_3gram

Number of frequent apps: 10


0                        com.android.incallui com.sonyericsson.android.socialphonebook com.android.incallui
1              com.android.incallui com.sonyericsson.android.socialphonebook com.sonyericsson.conversations
2                                    com.facebook.katana com.sonyericsson.conversations com.facebook.katana
3        com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox
4    com.sonyericsson.android.socialphonebook com.android.incallui com.sonyericsson.android.socialphonebook
5              com.sonyericsson.android.socialphonebook com.android.incallui com.sonyericsson.conversations
6              com.sonyericsson.conversations com.android.incallui com.sonyericsson.android.socialphonebook
7                        com.sonyericsson.conversations com.android.incallui com.sonyericsson.conversations
8                         com.sonyericsson.conversations com.facebook.katana com.sonyericsson.conversations
9              com.sonyerics

In [40]:
vectorizer_3_f5 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_f5=vectorizer_3_f5.fit_transform(corpus_top_500_factor_5_loadings.values)
chrome_component_3gram=pd.Series(vectorizer_3_f5.get_feature_names_out()[np.where((vectorization_3_f5>0).sum(axis=0)>=min_sup_5)[1]])
chrome_component_3gram.to_csv("3gram_chrome_component_minsup1.csv")
print("Number of frequent apps:",len(chrome_component_3gram))
chrome_component_3gram

Number of frequent apps: 4


0     com.android.chrome com.facebook.katana com.android.chrome
1            com.android.chrome com.whatsapp com.android.chrome
2    com.facebook.katana com.android.chrome com.facebook.katana
3                  com.whatsapp com.android.chrome com.whatsapp
dtype: object

In [41]:
vectorizer_3_f6 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_f6=vectorizer_3_f6.fit_transform(corpus_top_500_factor_6_loadings.values)
whatsapp_component_3gram=pd.Series(vectorizer_3_f5.get_feature_names_out()[np.where((vectorization_3_f5>0).sum(axis=0)>=min_sup_5)[1]])
whatsapp_component_3gram.to_csv("3gram_whatsapp_component_minsup1.csv")
print("Number of frequent apps:",len(whatsapp_component_3gram))
whatsapp_component_3gram

Number of frequent apps: 4


0     com.android.chrome com.facebook.katana com.android.chrome
1            com.android.chrome com.whatsapp com.android.chrome
2    com.facebook.katana com.android.chrome com.facebook.katana
3                  com.whatsapp com.android.chrome com.whatsapp
dtype: object

In [42]:
vectorizer_3_fr = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3),lowercase=False)
vectorization_3_fr=vectorizer_3_fr.fit_transform(corpus_top_500_factor_R_loadings.values)
email_component_3gram=pd.Series(vectorizer_3_fr.get_feature_names_out()[np.where((vectorization_3_fr>0).sum(axis=0)>=min_sup_r)[1]])
email_component_3gram.to_csv("3gram_email_component_minsup1.csv")
print("Number of frequent apps:",len(email_component_3gram))
email_component_3gram

Number of frequent apps: 3


0    com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox
1                                          com.sonymobile.email com.android.chrome com.sonymobile.email
2                       com.sonymobile.email com.google.android.googlequicksearchbox com.android.chrome
dtype: object

# 4-grams

In [43]:
vectorizer_4_f0 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_f0=vectorizer_4_f0.fit_transform(corpus_top_500_factor_0_loadings.values)
Facebook_component_4gram=pd.Series(vectorizer_4_f0.get_feature_names_out()[np.where((vectorization_4_f0>0).sum(axis=0)>=min_sup_0)[1]])
Facebook_component_4gram.to_csv("4gram_facebook_component_minsup1.csv")
print("Number of frequent apps:",len(Facebook_component_4gram))
Facebook_component_4gram

Number of frequent apps: 4


0    com.facebook.katana com.facebook.orca com.facebook.katana com.facebook.orca
1              com.facebook.katana com.whatsapp com.facebook.katana com.whatsapp
2    com.facebook.orca com.facebook.katana com.facebook.orca com.facebook.katana
3              com.whatsapp com.facebook.katana com.whatsapp com.facebook.katana
dtype: object

In [44]:
vectorizer_4_f1 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_f1=vectorizer_4_f1.fit_transform(corpus_top_500_factor_1_loadings.values)
google_component_4gram=pd.Series(vectorizer_4_f1.get_feature_names_out()[np.where((vectorization_4_f1>0).sum(axis=0)>=min_sup_1)[1]])
google_component_4gram.to_csv("4gram_google_component_minsup1.csv")
print("Number of frequent apps:",len(google_component_4gram))
google_component_4gram

Number of frequent apps: 3


0     com.android.chrome com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox
1    com.facebook.katana com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox
2     com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox com.android.chrome
dtype: object

In [45]:
vectorizer_4_f2 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_f2=vectorizer_4_f2.fit_transform(corpus_top_500_factor_2_loadings.values)
Weather_component_4gram=pd.Series(vectorizer_4_f2.get_feature_names_out()[np.where((vectorization_4_f2>0).sum(axis=0)>=min_sup_2)[1]])
Weather_component_4gram.to_csv("4gram_weather_component_minsup1.csv")
print("Number of frequent apps:",len(Weather_component_4gram))
Weather_component_4gram

Number of frequent apps: 0


Series([], dtype: object)

In [46]:
vectorizer_4_f3 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_f3=vectorizer_4_f3.fit_transform(corpus_top_500_factor_3_loadings.values)
social_multimedia_component_4gram=pd.Series(vectorizer_4_f3.get_feature_names_out()[np.where((vectorization_4_f3>0).sum(axis=0)>=min_sup_3)[1]])
social_multimedia_component_4gram.to_csv("4gram_social_multimedia_component_minsup1.csv")
print("Number of frequent apps:",len(social_multimedia_component_4gram))
social_multimedia_component_4gram

Number of frequent apps: 0


Series([], dtype: object)

In [47]:
vectorizer_4_f4 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_f4=vectorizer_4_f4.fit_transform(corpus_top_500_factor_4_loadings.values)
traditional_component_4gram=pd.Series(vectorizer_4_f4.get_feature_names_out()[np.where((vectorization_4_f4>0).sum(axis=0)>=min_sup_4)[1]])
traditional_component_4gram.to_csv("4gram_tradition_component_minsup1.csv")
print("Number of frequent apps:",len(traditional_component_4gram))
traditional_component_4gram

Number of frequent apps: 0


Series([], dtype: object)

In [48]:
vectorizer_4_f5 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_f5=vectorizer_4_f5.fit_transform(corpus_top_500_factor_5_loadings.values)
chrome_component_4gram=pd.Series(vectorizer_4_f5.get_feature_names_out()[np.where((vectorization_4_f5>0).sum(axis=0)>=min_sup_5)[1]])
chrome_component_4gram.to_csv("4gram_chrome_component_minsup1.csv")
print("Number of frequent apps:",len(chrome_component_4gram))
chrome_component_4gram

Number of frequent apps: 0


Series([], dtype: object)

In [49]:
vectorizer_4_f6 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_f6=vectorizer_4_f6.fit_transform(corpus_top_500_factor_6_loadings.values)
whatsapp_component_4gram=pd.Series(vectorizer_4_f5.get_feature_names_out()[np.where((vectorization_4_f5>0).sum(axis=0)>=min_sup_5)[1]])
whatsapp_component_4gram.to_csv("4gram_whatsapp_component_minsup1.csv")
print("Number of frequent apps:",len(whatsapp_component_4gram))
whatsapp_component_4gram

Number of frequent apps: 0


Series([], dtype: object)

In [50]:
vectorizer_4_fr = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4),lowercase=False)
vectorization_4_fr=vectorizer_4_fr.fit_transform(corpus_top_500_factor_R_loadings.values)
email_component_4gram=pd.Series(vectorizer_4_fr.get_feature_names_out()[np.where((vectorization_4_fr>0).sum(axis=0)>=min_sup_r)[1]])
email_component_4gram.to_csv("4gram_email_component_minsup1.csv")
print("Number of frequent apps:",len(email_component_4gram))
email_component_4gram

Number of frequent apps: 0


Series([], dtype: object)

# 5-grams

In [51]:
vectorizer_5_f0 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(5,5),lowercase=False)
vectorization_5_f0=vectorizer_5_f0.fit_transform(corpus_top_500_factor_0_loadings.values)
Facebook_component_5gram=pd.Series(vectorizer_5_f0.get_feature_names_out()[np.where((vectorization_5_f0>0).sum(axis=0)>=min_sup_0)[1]])
Facebook_component_5gram.to_csv("5gram_facebook_component_minsup1.csv")
print("Number of frequent apps:",len(Facebook_component_5gram))
Facebook_component_5gram

Number of frequent apps: 0


Series([], dtype: object)

In [52]:
vectorizer_5_f1 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(5,5),lowercase=False)
vectorization_5_f1=vectorizer_5_f1.fit_transform(corpus_top_500_factor_1_loadings.values)
google_component_5gram=pd.Series(vectorizer_5_f1.get_feature_names_out()[np.where((vectorization_5_f1>0).sum(axis=0)>=min_sup_1)[1]])
google_component_5gram.to_csv("5gram_google_component_minsup1.csv")
print("Number of frequent apps:",len(google_component_5gram))
google_component_5gram

Number of frequent apps: 1


0    com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox
dtype: object

In [53]:
vectorizer_5_f4 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(5,5),lowercase=False)
vectorization_5_f4=vectorizer_5_f4.fit_transform(corpus_top_500_factor_5_loadings.values)
traditional_component_5gram=pd.Series(vectorizer_5_f4.get_feature_names_out()[np.where((vectorization_5_f4>0).sum(axis=0)>=min_sup_4)[1]])
traditional_component_5gram.to_csv("5gram_tradition_component_minsup1.csv")
print("Number of frequent apps:",len(traditional_component_5gram))
traditional_component_4gram

Number of frequent apps: 0


Series([], dtype: object)

# 6-grams

In [54]:
vectorizer_5_f0 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(5,5),lowercase=False)
vectorization_5_f0=vectorizer_5_f0.fit_transform(corpus_top_500_factor_0_loadings.values)
Facebook_component_5gram=pd.Series(vectorizer_5_f0.get_feature_names_out()[np.where((vectorization_5_f0>0).sum(axis=0)>=min_sup_0)[1]])
Facebook_component_5gram.to_csv("5gram_facebook_component_minsup1.csv")
print("Number of frequent apps:",len(Facebook_component_5gram))
Facebook_component_5gram

Number of frequent apps: 0


Series([], dtype: object)

In [55]:
vectorizer_6_f1 = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(6,6),lowercase=False)
vectorization_6_f1=vectorizer_6_f1.fit_transform(corpus_top_500_factor_1_loadings.values)
google_component_6gram=pd.Series(vectorizer_6_f1.get_feature_names_out()[np.where((vectorization_6_f1>0).sum(axis=0)>=min_sup_1)[1]])
google_component_6gram.to_csv("6gram_google_component_minsup1.csv")
print("Number of frequent apps:",len(google_component_6gram))
google_component_6gram

Number of frequent apps: 0


Series([], dtype: object)

# VMSP

In [7]:
input_string_top_500_factor_0_loadings=data.loc[data.user_id.isin(top_500_factor_0_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")
input_string_top_500_factor_1_loadings=data.loc[data.user_id.isin(top_500_factor_1_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")
input_string_top_500_factor_2_loadings=data.loc[data.user_id.isin(top_500_factor_2_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")
input_string_top_500_factor_3_loadings=data.loc[data.user_id.isin(top_500_factor_3_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")
input_string_top_500_factor_4_loadings=data.loc[data.user_id.isin(top_500_factor_4_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")
input_string_top_500_factor_5_loadings=data.loc[data.user_id.isin(top_500_factor_5_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")
input_string_top_500_factor_6_loadings=data.loc[data.user_id.isin(top_500_factor_6_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")
input_string_top_500_factor_R_loadings=data.loc[data.user_id.isin(top_500_factor_R_loadings)].reset_index().groupby(['user_id','session']).app_code.apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")

In [8]:
np.savetxt('input_string_top_500_factor_0_loadings.txt', input_string_top_500_factor_0_loadings.values, fmt='%s')
np.savetxt('input_string_top_500_factor_1_loadings.txt', input_string_top_500_factor_1_loadings.values, fmt='%s')
np.savetxt('input_string_top_500_factor_2_loadings.txt', input_string_top_500_factor_2_loadings.values, fmt='%s')
np.savetxt('input_string_top_500_factor_3_loadings.txt', input_string_top_500_factor_3_loadings.values, fmt='%s')
np.savetxt('input_string_top_500_factor_4_loadings.txt', input_string_top_500_factor_4_loadings.values, fmt='%s')
np.savetxt('input_string_top_500_factor_5_loadings.txt', input_string_top_500_factor_5_loadings.values, fmt='%s')
np.savetxt('input_string_top_500_factor_6_loadings.txt', input_string_top_500_factor_6_loadings.values, fmt='%s')
np.savetxt('input_string_top_500_factor_R_loadings.txt', input_string_top_500_factor_R_loadings.values, fmt='%s')

In [9]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_0_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_facebook_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_facebook_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 76835 ms
 Frequent sequences count : 227
 Max memory (mb) : 14011.276428222656227
minsup 26088
Intersection count 1725 


                                               pattern    sup
0                                              [45327]  30016
1                                              [45284]  27794
2                                              [45246]  37101
3                                              [36529]  47204
4                                               [7655]  30804
..                                                 ...    ...
222          [19968, 19968, 7649, 19968, 19968, 19968]  30619
223           [19968, 7649, 19968, 7649, 19968, 19968]  28560
224           [19968, 7649, 19968, 19968, 7649, 19968]  27082
225           [19968, 19968, 7649, 19968, 7649, 19968]  29082
226  [19968, 19968, 19968, 19968, 19968, 19968, 19968]  30337

[227 r

In [10]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_1_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_google_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_google_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 47657 ms
 Frequent sequences count : 169
 Max memory (mb) : 10098.415115356445169
minsup 20562
Intersection count 1181 


                                            pattern    sup
0                                           [65472]  25846
1                                           [53996]  49851
2                                           [45846]  29508
3                                           [45327]  26121
4                                           [45284]  23743
..                                              ...    ...
164  [23949, 23949, 7649, 23949, 7649, 23949, 7649]  22445
165   [23949, 7649, 7649, 23949, 7649, 23949, 7649]  20816
166   [23949, 7649, 23949, 7649, 7649, 23949, 7649]  20832
167   [7649, 23949, 7649, 23949, 7649, 23949, 7649]  23138
168   [23949, 7649, 23949, 7649, 23949, 7649, 7649]  21072

[169 rows x 2 columns]


In [11]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_2_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_weather_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_weather_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 38974 ms
 Frequent sequences count : 154
 Max memory (mb) : 9396.947372436523154
minsup 19390
Intersection count 945 


                          pattern    sup
0                         [65472]  19771
1                         [53996]  27628
2                         [45846]  51245
3                         [45327]  28023
4                         [45284]  40622
..                            ...    ...
149  [19968, 19968, 19968, 19968]  26775
150   [19968, 19968, 7649, 19968]  22021
151   [19968, 7649, 19968, 19968]  21189
152    [23949, 7649, 23949, 7649]  27933
153    [19968, 7649, 19968, 7649]  21397

[154 rows x 2 columns]


In [12]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_3_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_socialmedia_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_socialmedia_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 178217 ms
 Frequent sequences count : 810
 Max memory (mb) : 3140.99422454834810
minsup 31361
Intersection count 6016 


                                                      pattern    sup
0                                                     [50264]  36200
1                                                     [45246]  33197
2                                                     [39356]  33879
3                                                     [36529]  40659
4                                                     [33357]  68120
..                                                        ...    ...
805         [44880, 27441, 44880, 27441, 44880, 44880, 27441]  32457
806         [44880, 44880, 27441, 44880, 27441, 44880, 27441]  32433
807         [44880, 27441, 44880, 44880, 27441, 44880, 27441]  32422
808         [27441, 44880, 27441, 44880, 27441, 44880, 27441] 

In [13]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_4_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_traditional_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_traditional_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 20519 ms
 Frequent sequences count : 127
 Max memory (mb) : 4514.04012298584127
minsup 14448
Intersection count 972 


                                 pattern    sup
0                                [53996]  18085
1                                [50519]  35651
2                                [45327]  23879
3                                [45284]  20341
4                                [45246]  23008
..                                   ...    ...
122           [7653, 45261, 7653, 45261]  19058
123         [19968, 19968, 45264, 19968]  15679
124         [19968, 45264, 19968, 19968]  15920
125           [23949, 7649, 23949, 7649]  17748
126  [45264, 19968, 45264, 19968, 45264]  14630

[127 rows x 2 columns]


In [14]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_5_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_chrome_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_chrome_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 48275 ms
 Frequent sequences count : 133
 Max memory (mb) : 13616.769630432129133
minsup 24708
Intersection count 719 


                               pattern    sup
0                              [65472]  26846
1                              [45846]  60394
2                              [45327]  28078
3                              [45284]  51665
4                              [45246]  59375
..                                 ...    ...
128          [19968, 7649, 7649, 7649]  33611
129          [7649, 7649, 7649, 19968]  29404
130  [53003, 7649, 53003, 7649, 53003]  28256
131   [7649, 53003, 7649, 53003, 7649]  26794
132     [7649, 7649, 7649, 7649, 7649]  41075

[133 rows x 2 columns]


In [15]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_6_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_whatsapp_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_whatsapp_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 102606 ms
 Frequent sequences count : 302
 Max memory (mb) : 2568.305595397949302
minsup 29710
Intersection count 2294 


                                                      pattern    sup
0                                                     [45327]  33614
1                                                     [45284]  53138
2                                                     [45246]  38729
3                                                     [31081]  29899
4                                                     [23941]  38576
..                                                        ...    ...
297         [53003, 19968, 53003, 19968, 53003, 19968, 53003]  41504
298         [19968, 53003, 53003, 19968, 53003, 19968, 53003]  30408
299         [19968, 53003, 19968, 53003, 19968, 53003, 19968]  32114
300         [53003, 19968, 53003, 19968, 53003, 19968, 19968]

In [16]:
spmf = Spmf("VMSP", input_filename="/home/s164574/motifs_and_competition_in_app_usage/input_string_top_500_factor_R_loadings.txt",
            output_filename="/home/s164574/motifs_and_competition_in_app_usage/output_VMSP_email_group_min_sup_1.txt",
            spmf_bin_location_dir="/home/s164574/motifs_and_competition_in_app_usage/",
            arguments=[0.01])
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output_VMSP_email_group_min_sup_1.csv")

>/home/s164574/motifs_and_competition_in_app_usage/spmf.jar
=============  VMSP v0.97 - STATISTICS =============
 Total time ~ 39680 ms
 Frequent sequences count : 135
 Max memory (mb) : 9774.336929321289135
minsup 21499
Intersection count 785 


                        pattern    sup
0                       [65472]  39722
1                       [45846]  26064
2                       [45327]  43810
3                       [45284]  31086
4                       [44880]  27969
..                          ...    ...
130   [7649, 45329, 7649, 7649]  23380
131   [7649, 7649, 45329, 7649]  22398
132   [45329, 7649, 7649, 7649]  25087
133  [7649, 23949, 7649, 23949]  22901
134  [23949, 7649, 23949, 7649]  30953

[135 rows x 2 columns]


## Translating results

In [17]:
app_dict=dict(zip(data.app_code, data.package_name))

### Facebook component
Have many revisites to facebook\
also revisits to other communication apps like whatsapp.\
Especially revisitis between whats app and facebook\
longest pattern is seven revisits to facebook\
Try an look at how many patterns consit only of whatsapp and facebook.
Or facebook and one other app.

In [18]:
output_facebook=pd.read_csv("output_VMSP_facebook_group_min_sup_1.csv",delimiter=';')
output_facebook['pattern']=output_facebook.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_facebook=output_facebook.sort_values(['sup'],ascending=False)
output_facebook['pattern_length']=output_facebook.pattern.apply(lambda l: len(l))
output_facebook.to_csv("output_VMSP_facebook_group_min_sup_1_translated.csv",index=False)

In [25]:
output_facebook=pd.read_csv("output_VMSP_facebook_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [19]:
print("longest frequent pattern:",output_facebook.pattern_length.max())

longest frequent pattern: 7


In [20]:
output_facebook.head(10)

,pattern,sup,pattern_length
71,"[com.facebook.katana, com.sonymobile.email, com.facebook.katana]",62394,3
30,"[com.sonyericsson.android.socialphonebook, com.facebook.katana]",56818,2
98,"[com.facebook.katana, com.google.android.gm, com.facebook.katana]",54852,3
40,"[com.android.incallui, com.sonyericsson.android.socialphonebook]",52955,2
3,[com.nianticlabs.pokemongo],47204,1
23,"[com.sonyericsson.conversations, com.google.android.googlequicksearchbox]",44349,2
68,"[com.facebook.katana, com.twitter.android, com.facebook.katana]",43065,3
69,"[com.sonymobile.email, com.facebook.katana, com.facebook.katana]",41795,3
5,[com.amazon.mShop.android.shopping],40557,1
88,"[com.facebook.katana, com.google.android.youtube, com.facebook.katana]",40532,3


### Google component
- google with a lot of other popular apps
- alot of back and forth between google and other apps
- also back and forth between comunications apps like whatsapp.
- also going back and forth between chrome, can be seen as revisited often with patterns of length 5. (not seen for facebook)
- there are 10 length 6 patterns and they all just describe using chrome and google. Same for the 9 length 7 patterns.

In [21]:
output_google=pd.read_csv("output_VMSP_google_group_min_sup_1.csv",delimiter=';')
output_google['pattern']=output_google.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_google=output_google.sort_values(['sup'],ascending=False)
output_google['pattern_length']=output_google.pattern.apply(lambda l: len(l))
output_google.to_csv("output_VMSP_google_group_min_sup_1_translated.csv",index=False)

In [61]:
test=output_google[output_google.pattern_length==7]

In [63]:
test[[1,2,3,4,5,6,7]]=test.pattern.apply(lambda l:str(l).replace('[','').replace(']','').replace('\'','')).str.split(',',expand=True)

/home/s164574/.conda/envs/myenv/lib/python3.8/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [64]:
test.to_csv("test.csv",index=False)

In [26]:
output_google=pd.read_csv("output_VMSP_google_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [22]:
print("longest frequent pattern:",output_google.pattern_length.max())

longest frequent pattern: 7


In [23]:
output_google.head(10)

,pattern,sup,pattern_length
35,"[com.sonyericsson.conversations, com.facebook.katana]",59175,2
44,"[com.sonyericsson.android.socialphonebook, com.android.incallui]",57740,2
5,[com.sony.nfx.app.sfrc],54661,1
1,[com.yahoo.mobile.client.android.mail],49851,1
17,"[com.sonyericsson.conversations, com.whatsapp]",48720,2
16,"[com.whatsapp, com.sonyericsson.conversations]",45526,2
6,[com.snapchat.android],45107,1
36,"[com.facebook.katana, com.sonyericsson.conversations]",44616,2
10,[com.google.android.apps.messaging],42596,1
135,"[com.google.android.googlequicksearchbox, com.android.chrome, com.google.android.googlequicksearchbox, com.facebook.katana]",42551,4


### Weather component
- do not have the xperia weather as a frequent app with the min. sup 1%
- back and forth between mesenger (length 3)?
- back and forth between google (length 3)?
- whatsapp and facebook also
- length 4 back and forth between facebook and whatsapp and back and forth between google and chrome. 
- so common patterns, so maybe it is more of a one use thing with the weather app, which is not used by other groups, but it is still only a small part of there overall usage.

In [24]:
output_weather=pd.read_csv("output_VMSP_weather_group_min_sup_1.csv",delimiter=';')
output_weather['pattern']=output_weather.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_weather=output_weather.sort_values(['sup'],ascending=False)
output_weather['pattern_length']=output_weather.pattern.apply(lambda l: len(l))
output_weather.to_csv("output_VMSP_weather_group_min_sup_1_translated.csv",index=False)

In [27]:
output_weather=pd.read_csv("output_VMSP_weather_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [25]:
print("longest frequent pattern:",output_weather.pattern_length.max())

longest frequent pattern: 4


In [26]:
output_weather.head(10)

,pattern,sup,pattern_length
2,[com.spotify.music],51245,1
43,"[com.sonymobile.email, com.facebook.katana]",50824,2
60,"[com.sonymobile.email, com.android.chrome]",49412,2
82,"[com.google.android.gm, com.android.chrome]",47257,2
138,"[com.android.chrome, com.android.chrome, com.android.chrome]",46572,3
130,"[com.facebook.katana, com.facebook.orca, com.facebook.katana]",46379,3
86,"[com.facebook.orca, com.android.chrome]",45968,2
9,[com.ebay.mobile],45102,1
73,"[com.google.android.gm, com.facebook.katana]",42844,2
95,"[com.whatsapp, com.android.chrome, com.whatsapp]",42342,3


### Social media - multimedia component
- a lot of patterns with instagram and snapchat being used with other popular apps.
- back and forth between facebook and messenger (length 4)
- revisiting messenger (length 5)
- back and forth between messenger, facebook, snapchat, instagram
- all 17 length 7 patterns are going between the snapchat and instagram
- longest pattern, 8 and is only snapchat. So snapchat as communication..?

In [27]:
output_socialmedia=pd.read_csv("output_VMSP_socialmedia_group_min_sup_1.csv",delimiter=';')
output_socialmedia['pattern']=output_socialmedia.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_socialmedia=output_socialmedia.sort_values(['sup'],ascending=False)
output_socialmedia['pattern_length']=output_socialmedia.pattern.apply(lambda l: len(l))
output_socialmedia.to_csv("output_VMSP_socialmedia_group_min_sup_1_translated.csv",index=False)

In [28]:
output_socialmedia=pd.read_csv("output_VMSP_socialmedia_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [28]:
print("longest frequent pattern:",output_socialmedia.pattern_length.max())

longest frequent pattern: 8


In [29]:
output_socialmedia.head(10)

,pattern,sup,pattern_length
27,"[com.spotify.music, com.instagram.android]",71328,2
6,[com.android.phone],69739,1
68,"[com.google.android.youtube, com.android.chrome]",69583,2
4,[com.microsoft.office.outlook],68120,1
291,"[com.android.chrome, com.android.chrome, com.android.chrome]",67927,3
41,"[com.facebook.katana, com.spotify.music]",65968,2
42,"[com.spotify.music, com.facebook.katana]",64326,2
29,"[com.facebook.orca, com.twitter.android]",64124,2
40,"[com.facebook.orca, com.spotify.music]",64107,2
147,"[com.whatsapp, com.instagram.android, com.facebook.katana]",63583,3


### Traditional communication component
- phonebook, conversations
- conversations x 4
- longest conversation -> facebook -> conversations ->facebook -> conversations

In [30]:
output_traditional=pd.read_csv("output_VMSP_traditional_group_min_sup_1.csv",delimiter=';')
output_traditional['pattern']=output_traditional.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_traditional=output_traditional.sort_values(['sup'],ascending=False)
output_traditional['pattern_length']=output_traditional.pattern.apply(lambda l: len(l))
output_traditional.to_csv("output_VMSP_traditional_group_min_sup_1_translated.csv",index=False)

In [29]:
output_traditional=pd.read_csv("output_VMSP_traditional_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [31]:
print("longest frequent pattern:",output_traditional.pattern_length.max())

longest frequent pattern: 5


In [32]:
output_traditional.head(10)

,pattern,sup,pattern_length
43,"[com.sonymobile.email, com.android.chrome]",48791,2
6,[com.google.android.youtube],48682,1
34,"[com.sonymobile.email, com.facebook.katana]",45697,2
9,[com.android.email],43416,1
77,"[com.sonyericsson.conversations, com.android.chrome, com.sonyericsson.conversations]",37116,3
1,[com.twitter.android],35651,1
19,"[com.sonyericsson.conversations, com.sonyericsson.album]",34719,2
40,"[com.whatsapp, com.android.chrome]",33471,2
35,"[com.facebook.katana, com.sonymobile.email]",33304,2
16,"[com.sonymobile.email, com.sonymobile.email]",31928,2


### Chrome component
- chrome with other apps
- whatsapp x4
- between chrome and facebook
- chrome x5
- whatsapp  chrome whatsapp chrome whatsapp

In [33]:
output_chrome=pd.read_csv("output_VMSP_chrome_group_min_sup_1.csv",delimiter=';')
output_chrome['pattern']=output_chrome.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_chrome=output_chrome.sort_values(['sup'],ascending=False)
output_chrome['pattern_length']=output_chrome.pattern.apply(lambda l: len(l))
output_chrome.to_csv("output_VMSP_chrome_group_min_sup_1_translated.csv",index=False)

In [30]:
output_chrome=pd.read_csv("output_VMSP_chrome_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [34]:
print("longest frequent pattern:",output_chrome.pattern_length.max())

longest frequent pattern: 5


In [35]:
output_chrome.head(10)

,pattern,sup,pattern_length
1,[com.spotify.music],60394,1
15,"[com.sonyericsson.conversations, com.whatsapp]",59466,2
4,[com.sony.nfx.app.sfrc],59375,1
42,"[com.sonyericsson.android.socialphonebook, com.android.incallui]",57686,2
102,"[com.android.chrome, com.google.android.googlequicksearchbox, com.android.chrome]",57443,3
14,"[com.whatsapp, com.sonyericsson.conversations]",55319,2
109,"[com.android.chrome, com.facebook.orca, com.android.chrome]",53669,3
36,"[com.facebook.katana, com.sonyericsson.conversations]",53382,2
3,[com.sonyericsson.music],51665,1
58,"[com.microsoft.office.outlook, com.android.chrome]",49757,2


### Whatsapp component
- whatsapp revisited
- whatsapp with facebook and chrome
- length 7 is between whatsapp and facebook
- 8 x whatsapp


In [36]:
output_whatsapp=pd.read_csv("output_VMSP_whatsapp_group_min_sup_1.csv",delimiter=';')
output_whatsapp['pattern']=output_whatsapp.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_whatsapp=output_whatsapp.sort_values(['sup'],ascending=False)
output_whatsapp['pattern_length']=output_whatsapp.pattern.apply(lambda l: len(l))
output_whatsapp.to_csv("output_VMSP_whatsapp_group_min_sup_1_translated.csv",index=False)

In [31]:
output_whatsapp=pd.read_csv("output_VMSP_whatsapp_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [37]:
print("longest frequent pattern:",output_whatsapp.pattern_length.max())

longest frequent pattern: 8


In [38]:
output_whatsapp.head(10)

,pattern,sup,pattern_length
15,"[com.whatsapp, com.spotify.music]",58780,2
14,"[com.spotify.music, com.whatsapp]",57621,2
56,"[com.whatsapp, com.sonymobile.email, com.whatsapp]",56341,3
83,"[com.whatsapp, com.android.incallui, com.whatsapp]",55739,3
63,"[com.whatsapp, com.google.android.apps.maps, com.whatsapp]",53350,3
1,[com.sonyericsson.music],53138,1
10,[bbc.mobile.news.uk],50230,1
37,"[com.google.android.gm, com.google.android.gm]",48710,2
32,"[com.android.chrome, com.sonyericsson.conversations]",48295,2
48,"[com.facebook.orca, com.android.chrome]",46924,2


### Email component
- whatsapp whatsapp whatsapp (like the others)
- patterns between popular apps
- sonyemail with chrome or facebook (length 4)
- sonyemail , but not as many revisitis - maybe because it is slower communication...

In [39]:
output_email=pd.read_csv("output_VMSP_email_group_min_sup_1.csv",delimiter=';')
output_email['pattern']=output_email.pattern.apply(lambda l: [app_dict[int(i)] for i in ast.literal_eval(l)])
output_email=output_email.sort_values(['sup'],ascending=False)
output_email['pattern_length']=output_email.pattern.apply(lambda l: len(l))
output_email.to_csv("output_VMSP_email_group_min_sup_1_translated.csv",index=False)

In [32]:
output_email=pd.read_csv("output_VMSP_email_group_min_sup_1_translated.csv",converters={"pattern": literal_eval})

In [354]:
test=output_email[output_email.pattern_length==4]
test[[1,2,3,4]]=test.pattern.apply(lambda l:str(l).replace('[','').replace(']','')\
                                                                                .replace('\'','')).str.split(',',expand=True)

test.to_csv('test.csv')

In [40]:
print("longest frequent pattern:",output_email.pattern_length.max())

longest frequent pattern: 4


In [41]:
output_email.head(10)

,pattern,sup,pattern_length
8,[com.google.android.youtube],65520,1
17,"[com.sonyericsson.conversations, com.whatsapp]",56114,2
18,"[com.whatsapp, com.sonyericsson.conversations]",52452,2
117,"[com.facebook.katana, com.android.chrome, com.facebook.katana]",50173,3
15,[com.amazon.mShop.android.shopping],48695,1
105,"[com.facebook.katana, com.sonymobile.email, com.android.chrome]",48237,3
107,"[com.sonymobile.email, com.facebook.katana, com.android.chrome]",47781,3
63,"[com.whatsapp, com.sonymobile.email, com.whatsapp]",47428,3
113,"[com.facebook.katana, com.facebook.katana, com.facebook.katana]",44264,3
39,"[com.sonyericsson.conversations, com.google.android.googlequicksearchbox]",44116,2


## comparing results

There is quit a few number of patterns found for all groups so to compare them we look how they intersect.

In [34]:
facebook_app_count=pd.Series(output_facebook.pattern.sum()).value_counts()
google_app_count=pd.Series(output_google.pattern.sum()).value_counts()
weather_app_count=pd.Series(output_weather.pattern.sum()).value_counts()
socialmedia_app_count=pd.Series(output_socialmedia.pattern.sum()).value_counts()
traditional_app_count=pd.Series(output_traditional.pattern.sum()).value_counts()
chrome_app_count=pd.Series(output_chrome.pattern.sum()).value_counts()
whatsapp_app_count=pd.Series(output_whatsapp.pattern.sum()).value_counts()
email_app_count=pd.Series(output_email.pattern.sum()).value_counts()

In [35]:
common_frequent_apps=set(facebook_app_count.index).intersection(set(google_app_count.index)).intersection(set(weather_app_count.index)).\
intersection(set(socialmedia_app_count.index)).intersection(set(traditional_app_count.index)).intersection(set(chrome_app_count.index)).\
intersection(set(whatsapp_app_count.index)).intersection(set(email_app_count.index))

In [39]:
pd.DataFrame(facebook_app_count).rename(columns={0:'facebook'}).join(pd.DataFrame(google_app_count).rename(columns={0:'google'}),how='outer')\
.join(pd.DataFrame(weather_app_count).rename(columns={0:'weather'}),how='outer').join(pd.DataFrame(socialmedia_app_count).rename(columns={0:'socialmedia'}),how='outer')\
.join(pd.DataFrame(traditional_app_count).rename(columns={0:'traditional'}),how='outer').join(pd.DataFrame(chrome_app_count).rename(columns={0:'chrome'}),how='outer')\
.join(pd.DataFrame(whatsapp_app_count).rename(columns={0:'whatsapp'}),how='outer').join(pd.DataFrame(email_app_count).rename(columns={0:'email'}),how='outer')

,facebook,google,weather,socialmedia,traditional,chrome,whatsapp,email
bbc.mobile.news.uk,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
bbc.mobile.weather,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0
com.amazon.mShop.android.shopping,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0
com.android.chrome,97.0,131.0,57.0,152.0,47.0,123.0,136.0,63.0
com.android.email,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
com.android.incallui,5.0,5.0,9.0,1.0,28.0,5.0,10.0,5.0
com.android.phone,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
com.dailymail.online,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
com.ebay.mobile,1.0,2.0,1.0,NaN,1.0,2.0,1.0,1.0
com.facebook.katana,378.0,48.0,88.0,658.0,50.0,51.0,197.0,50.0


There is a large overlap between the apps, and there is no apps which are only found to be frequent for one of the groups.\
However we can see that it is very different how many times and app occur.

We look at how many maximal patterns the app occur in

In [42]:
facebook_app_count=pd.Series(output_facebook.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_facebook.shape[0]
google_app_count=pd.Series(output_google.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_google.shape[0]
weather_app_count=pd.Series(output_weather.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_weather.shape[0]
socialmedia_app_count=pd.Series(output_socialmedia.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_socialmedia.shape[0]
traditional_app_count=pd.Series(output_traditional.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_traditional.shape[0]
chrome_app_count=pd.Series(output_chrome.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_chrome.shape[0]
whatsapp_app_count=pd.Series(output_whatsapp.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_whatsapp.shape[0]
email_app_count=pd.Series(output_email.pattern.apply(lambda l: list(set(l))).sum()).value_counts()/output_email.shape[0]

common_frequent_apps=set(facebook_app_count.index).intersection(set(google_app_count.index)).intersection(set(weather_app_count.index)).\
intersection(set(socialmedia_app_count.index)).intersection(set(traditional_app_count.index)).intersection(set(chrome_app_count.index)).\
intersection(set(whatsapp_app_count.index)).intersection(set(email_app_count.index))

pd.DataFrame(facebook_app_count).rename(columns={0:'facebook'}).join(pd.DataFrame(google_app_count).rename(columns={0:'google'}),how='outer')\
.join(pd.DataFrame(weather_app_count).rename(columns={0:'weather'}),how='outer').join(pd.DataFrame(socialmedia_app_count).rename(columns={0:'socialmedia'}),how='outer')\
.join(pd.DataFrame(traditional_app_count).rename(columns={0:'traditional'}),how='outer').join(pd.DataFrame(chrome_app_count).rename(columns={0:'chrome'}))\
.join(pd.DataFrame(whatsapp_app_count).rename(columns={0:'whatsapp'}),how='outer').join(pd.DataFrame(email_app_count).rename(columns={0:'email'}),how='outer').sort_values('facebook',ascending=False)

,facebook,google,weather,socialmedia,traditional,chrome,whatsapp,email
com.facebook.katana,0.740088,0.224852,0.402597,0.490123,0.299213,0.293233,0.410596,0.281481
com.android.chrome,0.317181,0.485207,0.305195,0.165432,0.291339,0.548872,0.337748,0.348148
com.facebook.orca,0.277533,0.029586,0.103896,0.354321,0.086614,0.097744,0.072848,0.044444
com.whatsapp,0.259912,0.183432,0.266234,0.149383,0.141732,0.270677,0.768212,0.222222
com.google.android.googlequicksearchbox,0.140969,0.650888,0.116883,0.085185,0.118110,0.060150,0.105960,0.125926
com.sonyericsson.conversations,0.140969,0.106509,0.162338,0.046914,0.425197,0.105263,0.062914,0.148148
com.instagram.android,0.070485,0.029586,0.051948,0.503704,0.031496,0.052632,0.198675,0.037037
com.sonymobile.email,0.048458,0.047337,0.058442,0.006173,0.102362,0.052632,0.026490,0.466667
com.sonyericsson.album,0.039648,0.041420,0.045455,0.019753,0.055118,0.045113,0.052980,0.044444
com.google.android.gm,0.039648,0.088757,0.058442,0.011111,0.039370,0.067669,0.039735,0.007407


In [67]:
pd.DataFrame(facebook_app_count).rename(columns={0:'facebook'}).join(pd.DataFrame(google_app_count).rename(columns={0:'google'}),how='outer')\
.join(pd.DataFrame(weather_app_count).rename(columns={0:'weather'}),how='outer').join(pd.DataFrame(socialmedia_app_count).rename(columns={0:'socialmedia'}),how='outer')\
.join(pd.DataFrame(traditional_app_count).rename(columns={0:'traditional'}),how='outer').join(pd.DataFrame(chrome_app_count).rename(columns={0:'chrome'}))\
.join(pd.DataFrame(whatsapp_app_count).rename(columns={0:'whatsapp'}),how='outer').join(pd.DataFrame(email_app_count).rename(columns={0:'email'}),how='outer').sort_values('email',ascending=False)

,facebook,google,weather,socialmedia,traditional,chrome,whatsapp,email
com.sonymobile.email,0.048458,0.047337,0.058442,0.006173,0.102362,0.052632,0.026490,0.466667
com.android.chrome,0.317181,0.485207,0.305195,0.165432,0.291339,0.548872,0.337748,0.348148
com.facebook.katana,0.740088,0.224852,0.402597,0.490123,0.299213,0.293233,0.410596,0.281481
com.whatsapp,0.259912,0.183432,0.266234,0.149383,0.141732,0.270677,0.768212,0.222222
com.sonyericsson.conversations,0.140969,0.106509,0.162338,0.046914,0.425197,0.105263,0.062914,0.148148
com.google.android.googlequicksearchbox,0.140969,0.650888,0.116883,0.085185,0.118110,0.060150,0.105960,0.125926
com.sonyericsson.album,0.039648,0.041420,0.045455,0.019753,0.055118,0.045113,0.052980,0.044444
com.twitter.android,0.017621,0.023669,0.032468,0.030864,0.007874,0.045113,0.023179,0.044444
com.facebook.orca,0.277533,0.029586,0.103896,0.354321,0.086614,0.097744,0.072848,0.044444
com.android.incallui,0.022026,0.029586,0.051948,0.001235,0.173228,0.037594,0.029801,0.037037


In [303]:
pd.DataFrame(facebook_app_count).rename(columns={0:'facebook'}).join(pd.DataFrame(google_app_count).rename(columns={0:'google'}),how='outer')\
.join(pd.DataFrame(weather_app_count).rename(columns={0:'weather'}),how='outer').join(pd.DataFrame(socialmedia_app_count).rename(columns={0:'socialmedia'}),how='outer')\
.join(pd.DataFrame(traditional_app_count).rename(columns={0:'traditional'}),how='outer').join(pd.DataFrame(chrome_app_count).rename(columns={0:'chrome'}))\
.join(pd.DataFrame(whatsapp_app_count).rename(columns={0:'whatsapp'}),how='outer').join(pd.DataFrame(email_app_count).rename(columns={0:'email'}),how='outer').sort_values('weather',ascending=False)

,facebook,google,weather,socialmedia,traditional,chrome,whatsapp,email
com.facebook.katana,0.740088,0.224852,0.402597,0.490123,0.299213,0.293233,0.410596,0.281481
com.android.chrome,0.317181,0.485207,0.305195,0.165432,0.291339,0.548872,0.337748,0.348148
com.whatsapp,0.259912,0.183432,0.266234,0.149383,0.141732,0.270677,0.768212,0.222222
com.sonyericsson.conversations,0.140969,0.106509,0.162338,0.046914,0.425197,0.105263,0.062914,0.148148
com.google.android.googlequicksearchbox,0.140969,0.650888,0.116883,0.085185,0.118110,0.060150,0.105960,0.125926
com.facebook.orca,0.277533,0.029586,0.103896,0.354321,0.086614,0.097744,0.072848,0.044444
com.sonymobile.email,0.048458,0.047337,0.058442,0.006173,0.102362,0.052632,0.026490,0.466667
com.google.android.gm,0.039648,0.088757,0.058442,0.011111,0.039370,0.067669,0.039735,0.007407
com.android.incallui,0.022026,0.029586,0.051948,0.001235,0.173228,0.037594,0.029801,0.037037
com.instagram.android,0.070485,0.029586,0.051948,0.503704,0.031496,0.052632,0.198675,0.037037


In [65]:
pd.DataFrame(facebook_app_count).rename(columns={0:'facebook'}).join(pd.DataFrame(google_app_count).rename(columns={0:'google'}),how='outer')\
.join(pd.DataFrame(weather_app_count).rename(columns={0:'weather'}),how='outer').join(pd.DataFrame(socialmedia_app_count).rename(columns={0:'socialmedia'}),how='outer')\
.join(pd.DataFrame(traditional_app_count).rename(columns={0:'traditional'}),how='outer').join(pd.DataFrame(chrome_app_count).rename(columns={0:'chrome'}))\
.join(pd.DataFrame(whatsapp_app_count).rename(columns={0:'whatsapp'}),how='outer').join(pd.DataFrame(email_app_count).rename(columns={0:'email'}),how='outer').sort_values('socialmedia',ascending=False)

,facebook,google,weather,socialmedia,traditional,chrome,whatsapp,email
com.snapchat.android,0.030837,0.005917,0.038961,0.590123,0.007874,0.015038,0.069536,0.007407
com.instagram.android,0.070485,0.029586,0.051948,0.503704,0.031496,0.052632,0.198675,0.037037
com.facebook.katana,0.740088,0.224852,0.402597,0.490123,0.299213,0.293233,0.410596,0.281481
com.facebook.orca,0.277533,0.029586,0.103896,0.354321,0.086614,0.097744,0.072848,0.044444
com.android.chrome,0.317181,0.485207,0.305195,0.165432,0.291339,0.548872,0.337748,0.348148
com.whatsapp,0.259912,0.183432,0.266234,0.149383,0.141732,0.270677,0.768212,0.222222
com.google.android.googlequicksearchbox,0.140969,0.650888,0.116883,0.085185,0.118110,0.060150,0.105960,0.125926
com.sonyericsson.conversations,0.140969,0.106509,0.162338,0.046914,0.425197,0.105263,0.062914,0.148148
com.google.android.youtube,0.017621,0.041420,0.032468,0.046914,0.007874,0.052632,0.019868,0.007407
com.twitter.android,0.017621,0.023669,0.032468,0.030864,0.007874,0.045113,0.023179,0.044444


Some differences can be observed. Even though Facebook is part of many patterns accross the three it is part of the most patterns in the facebook group. Which further indicate that this is a group of user with a high facebook use. As expected it is by far the google group, which have a  high number of patterns with the google app. Social media - multimedia have a high number with messenger, but really stand out with the instagram and snapchat. Traditional stands out with conversations and call. Chrome with chrome, whatsapp with whatsapp and email with sony email.

To find out the differences we look at patterns that doesn't occur in all

In [172]:
test=output_chrome.copy()
test['test']=output_chrome.pattern.apply(lambda l:str(l).replace('[','').replace(']','').replace('\'',''))

In [167]:
test2=output_facebook.pattern.apply(lambda l:str(l).replace('[','').replace(']','').replace('\'','').replace(',',''))

In [189]:
type(output_facebook[output_facebook.pattern_length==1])

pandas.core.frame.DataFrame

In [217]:
facebook=output_facebook.copy()
facebook=facebook.drop(columns=['sup'])
facebook['facebook']=True
google=output_google.copy()
google=google.drop(columns=['sup'])
google['google']=True
weather=output_weather.copy()
weather=weather.drop(columns=['sup'])
weather['weather']=True
social=output_socialmedia.copy()
social=social.drop(columns=['sup'])
social['social']=True
traditional=output_traditional.copy()
traditional=traditional.drop(columns=['sup'])
traditional['traditional']=True
chrome=output_chrome.copy()
chrome=chrome.drop(columns=['sup'])
chrome['chrome']=True
whatsapp=output_whatsapp.copy()
whatsapp=whatsapp.drop(columns=['sup'])
whatsapp['whatsapp']=True
email=output_email.copy()
email=email.drop(columns=['sup'])
email['email']=True
facebook['pattern']=facebook['pattern'].apply(lambda l:str(l))
google['pattern']=google['pattern'].apply(lambda l:str(l))
weather['pattern']=weather['pattern'].apply(lambda l:str(l))
social['pattern']=social['pattern'].apply(lambda l:str(l))
traditional['pattern']=traditional['pattern'].apply(lambda l:str(l))
chrome['pattern']=chrome['pattern'].apply(lambda l:str(l))
whatsapp['pattern']=whatsapp['pattern'].apply(lambda l:str(l))
email['pattern']=email['pattern'].apply(lambda l:str(l))

In [226]:
facebook[facebook.pattern_length==1].merge(google[google.pattern_length==1],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==1],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==1],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==1],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==1],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==1],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==1],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,pattern,pattern_length,facebook,google,weather,social,traditional,chrome,whatsapp,email
0,['com.nianticlabs.pokemongo'],1,True,True,True,True,False,True,False,True
1,['com.amazon.mShop.android.shopping'],1,True,True,True,False,False,True,True,True
2,['com.sony.nfx.app.sfrc'],1,True,True,True,True,True,True,True,False
3,['com.android.phone'],1,True,True,True,True,True,True,True,False
4,['com.sonymobile.calendar'],1,True,True,True,False,True,True,True,True
5,['com.sonyericsson.music'],1,True,True,True,False,True,True,True,True
6,['com.yahoo.mobile.client.android.mail'],1,False,True,True,False,True,False,False,False
7,['com.snapchat.android'],1,False,True,False,False,False,False,False,True
8,['com.google.android.apps.messaging'],1,False,True,False,True,False,True,False,True
9,['com.google.android.apps.photos'],1,False,True,False,False,False,False,True,True


In [228]:
facebook[facebook.pattern_length==2].merge(google[google.pattern_length==2],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==2],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==2],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==2],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==2],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==2],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==2],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,pattern,pattern_length,facebook,google,weather,social,traditional,chrome,whatsapp,email
0,"['com.sonyericsson.android.socialphonebook', 'com.facebook.katana']",2,True,False,True,False,False,False,False,False
1,"['com.android.incallui', 'com.sonyericsson.android.socialphonebook']",2,True,True,False,False,False,True,False,False
2,"['com.sonyericsson.conversations', 'com.google.android.googlequicksearchbox']",2,True,False,False,True,False,False,True,True
3,"['com.google.android.googlequicksearchbox', 'com.sonyericsson.conversations']",2,True,False,True,True,False,False,False,True
4,"['com.whatsapp', 'com.instagram.android']",2,True,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
196,"['com.instagram.android', 'com.sonymobile.email']",2,False,False,False,False,False,False,False,True
197,"['com.android.incallui', 'com.sonymobile.email']",2,False,False,False,False,False,False,False,True
198,"['com.sony.nfx.app.sfrc', 'com.sonymobile.email']",2,False,False,False,False,False,False,False,True
199,"['com.google.android.gm', 'com.sonymobile.email']",2,False,False,False,False,False,False,False,True


In [229]:
facebook[facebook.pattern_length==3].merge(google[google.pattern_length==3],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==3],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==3],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==3],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==3],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==3],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==3],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,pattern,pattern_length,facebook,google,weather,social,traditional,chrome,whatsapp,email
0,"['com.facebook.katana', 'com.sonymobile.email', 'com.facebook.katana']",3,True,False,False,False,False,False,False,False
1,"['com.facebook.katana', 'com.google.android.gm', 'com.facebook.katana']",3,True,False,False,False,False,False,False,False
2,"['com.facebook.katana', 'com.twitter.android', 'com.facebook.katana']",3,True,False,False,True,False,False,False,False
3,"['com.sonymobile.email', 'com.facebook.katana', 'com.facebook.katana']",3,True,False,False,False,False,False,False,False
4,"['com.facebook.katana', 'com.google.android.youtube', 'com.facebook.katana']",3,True,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
442,"['com.sonyericsson.conversations', 'com.android.chrome', 'com.sonymobile.email']",3,False,False,False,False,False,False,False,True
443,"['com.android.chrome', 'com.sonymobile.email', 'com.google.android.googlequicksearchbox']",3,False,False,False,False,False,False,False,True
444,"['com.sonymobile.email', 'com.sonymobile.email', 'com.google.android.googlequicksearchbox']",3,False,False,False,False,False,False,False,True
445,"['com.sonymobile.email', 'com.facebook.katana', 'com.facebook.orca']",3,False,False,False,False,False,False,False,True


In [230]:
facebook[facebook.pattern_length==4].merge(google[google.pattern_length==4],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==4],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==4],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==4],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==4],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==4],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==4],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,pattern,pattern_length,facebook,google,weather,social,traditional,chrome,whatsapp,email
0,"['com.facebook.katana', 'com.instagram.android', 'com.facebook.katana', 'com.facebook.katana']",4,True,False,False,False,False,False,False,False
1,"['com.facebook.katana', 'com.facebook.katana', 'com.instagram.android', 'com.facebook.katana']",4,True,False,False,False,False,False,False,False
2,"['com.whatsapp', 'com.facebook.katana', 'com.android.chrome', 'com.facebook.katana']",4,True,False,False,False,False,False,False,False
3,"['com.google.android.googlequicksearchbox', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana']",4,True,False,False,False,False,False,False,False
4,"['com.sonyericsson.conversations', 'com.facebook.katana', 'com.facebook.katana', 'com.sonyericsson.conversations']",4,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
439,"['com.android.chrome', 'com.sonymobile.email', 'com.android.chrome', 'com.android.chrome']",4,False,False,False,False,False,False,False,True
440,"['com.sonymobile.email', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox']",4,False,False,False,False,False,False,False,True
441,"['com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox']",4,False,False,False,False,False,False,False,True
442,"['com.android.chrome', 'com.android.chrome', 'com.sonymobile.email', 'com.android.chrome']",4,False,False,False,False,False,False,False,True


In [232]:
facebook[facebook.pattern_length==5].merge(google[google.pattern_length==5],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==5],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==5],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==5],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==5],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==5],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==5],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,pattern,pattern_length,facebook,google,weather,social,traditional,chrome,whatsapp,email
0,"['com.facebook.katana', 'com.facebook.orca', 'com.facebook.katana', 'com.android.chrome', 'com.facebook.katana']",5,True,False,False,False,False,False,False,False
1,"['com.facebook.katana', 'com.sonyericsson.conversations', 'com.facebook.katana', 'com.sonyericsson.conversations', 'com.facebook.katana']",5,True,False,False,False,False,False,False,False
2,"['com.facebook.orca', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.orca']",5,True,False,False,True,False,False,False,False
3,"['com.facebook.katana', 'com.android.chrome', 'com.facebook.katana', 'com.facebook.orca', 'com.facebook.katana']",5,True,False,False,False,False,False,False,False
4,"['com.facebook.katana', 'com.sonyericsson.conversations', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana']",5,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
271,"['com.whatsapp', 'com.android.chrome', 'com.facebook.katana', 'com.whatsapp', 'com.whatsapp']",5,False,False,False,False,False,False,True,False
272,"['com.whatsapp', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.whatsapp', 'com.whatsapp']",5,False,False,False,False,False,False,True,False
273,"['com.instagram.android', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp', 'com.instagram.android']",5,False,False,False,False,False,False,True,False
274,"['com.whatsapp', 'com.facebook.katana', 'com.whatsapp', 'com.android.chrome', 'com.facebook.katana']",5,False,False,False,False,False,False,True,False


In [233]:
facebook[facebook.pattern_length==6].merge(google[google.pattern_length==6],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==6],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==6],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==6],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==6],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==6],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==6],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,pattern,pattern_length,facebook,google,weather,social,traditional,chrome,whatsapp,email
0,"['com.whatsapp', 'com.facebook.katana', 'com.whatsapp', 'com.facebook.katana', 'com.whatsapp', 'com.facebook.katana']",6,True,False,False,False,False,False,False,False
1,"['com.facebook.katana', 'com.whatsapp', 'com.facebook.katana', 'com.whatsapp', 'com.facebook.katana', 'com.whatsapp']",6,True,False,False,False,False,False,False,False
2,"['com.facebook.katana', 'com.facebook.orca', 'com.facebook.katana', 'com.facebook.orca', 'com.facebook.katana', 'com.facebook.katana']",6,True,False,False,False,False,False,False,False
3,"['com.facebook.katana', 'com.facebook.katana', 'com.facebook.orca', 'com.facebook.katana', 'com.facebook.orca', 'com.facebook.katana']",6,True,False,False,False,False,False,False,False
4,"['com.facebook.katana', 'com.whatsapp', 'com.facebook.katana', 'com.whatsapp', 'com.facebook.katana', 'com.facebook.katana']",6,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
116,"['com.whatsapp', 'com.instagram.android', 'com.whatsapp', 'com.whatsapp', 'com.instagram.android', 'com.whatsapp']",6,False,False,False,False,False,False,True,False
117,"['com.whatsapp', 'com.android.chrome', 'com.whatsapp', 'com.android.chrome', 'com.android.chrome', 'com.whatsapp']",6,False,False,False,False,False,False,True,False
118,"['com.whatsapp', 'com.android.chrome', 'com.android.chrome', 'com.whatsapp', 'com.android.chrome', 'com.whatsapp']",6,False,False,False,False,False,False,True,False
119,"['com.facebook.katana', 'com.facebook.katana', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp']",6,False,False,False,False,False,False,True,False


In [234]:
facebook[facebook.pattern_length==7].merge(google[google.pattern_length==7],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==7],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==7],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==7],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==7],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==7],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==7],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,pattern,pattern_length,facebook,google,weather,social,traditional,chrome,whatsapp,email
0,"['com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana', 'com.facebook.katana']",7,True,False,False,False,False,False,False,False
1,"['com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox']",7,False,True,False,False,False,False,False,False
2,"['com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.google.android.googlequicksearchbox', 'com.android.chrome']",7,False,True,False,False,False,False,False,False
3,"['com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome']",7,False,True,False,False,False,False,False,False
4,"['com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome']",7,False,True,False,False,False,False,False,False
5,"['com.google.android.googlequicksearchbox', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome']",7,False,True,False,False,False,False,False,False
6,"['com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.android.chrome']",7,False,True,False,False,False,False,False,False
7,"['com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome']",7,False,True,False,False,False,False,False,False
8,"['com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.android.chrome']",7,False,True,False,False,False,False,False,False
9,"['com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox', 'com.google.android.googlequicksearchbox', 'com.google.android.googlequicksearchbox', 'com.android.chrome', 'com.google.android.googlequicksearchbox']",7,False,True,False,False,False,False,False,False


In [235]:
facebook[facebook.pattern_length==8].merge(google[google.pattern_length==8],on=['pattern','pattern_length'],how='outer').\
merge(weather[weather.pattern_length==8],on=['pattern','pattern_length'],how='outer').\
merge(social[social.pattern_length==8],on=['pattern','pattern_length'],how='outer').\
merge(traditional[traditional.pattern_length==8],on=['pattern','pattern_length'],how='outer').\
merge(chrome[chrome.pattern_length==8],on=['pattern','pattern_length'],how='outer').\
merge(whatsapp[whatsapp.pattern_length==8],on=['pattern','pattern_length'],how='outer').\
merge(email[email.pattern_length==8],on=['pattern','pattern_length'],how='outer').replace(np.nan,False)

,facebook,google,weather,pattern,pattern_length,social,traditional,chrome,whatsapp,email
0,False,False,False,"['com.snapchat.android', 'com.snapchat.android', 'com.snapchat.android', 'com.snapchat.android', 'com.snapchat.android', 'com.snapchat.android', 'com.snapchat.android', 'com.snapchat.android']",8,True,False,False,False,False
1,False,False,False,"['com.whatsapp', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp', 'com.whatsapp']",8,False,False,False,True,False
